In [2]:
import pandas as pd
import numpy as np

**import directly from google sheets using this info**

https://skills.ai/blog/import-google-sheets-to-pandas/#:~:text=2%3A%20Reading%20Data%20from%20a%20Public%20Google%20Sheet%20into%20Pandas&text=Pandas%20can%20directly%20read%20CSV,format%3Dcsv%26gid%3D%7Bsheet_id%7D%20.

In [322]:
'''
fp_df is a dataframe of odds of each driver winning the race, as well as .
this may not be the best way to evaluate odds of finishing in each position, but its what i'm starting with
'''

url = 'https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid=0'
fp_df = pd.read_csv(url)

In [323]:
fp_df

,Team,Driver,odds_p1_bahrain,fp1_bahrain,fp2_bahrain,fp3_bahrain,quali_predicted_bahrain,qualifying_bahrain
0,Red Bull Racing-RBPT,Max Verstappen,-250,6,6,3,1,1
1,Red Bull Racing-RBPT,Sergio Perez,1400,12,10,8,6,5
2,Mercedes,Lewis Hamilton,800,9,1,12,3,9
3,Mercedes,George Russell,1400,7,2,6,4,3
4,Ferrari,Charles Leclerc,1100,8,9,4,2,2
5,Ferrari,Carlos Sainz Jr.,2200,11,4,1,5,4
6,McLaren-Mercedes,Oscar Piastri,2800,3,5,7,9,8
7,McLaren-Mercedes,Lando Norris,2000,2,20,5,7,7
8,Aston Martin Aramco-Mercedes,Fernando Alonso,2200,5,3,2,8,6
9,Aston Martin Aramco-Mercedes,Lance Stroll,20000,14,8,10,12,12


In [313]:
def driver_eval_fp_func(df, track):
    '''
    this function 
    '''
    sessions = [x for x in df.columns if 'fp' in x and track in x]
    print(f'There are {len(sessions)} practice sessions in this dataframe for {track.capitalize()} in 2024.')
    
    df_session_cols = []
    for i in range(len(sessions)):
        df_session_cols.append(f'FP{i+1} Results')
    
    df_spread_cols = ['FP1-FP2 Change', 'FP2-FP3 Change', 'FP1-FP3 Change']
    
    driver_eval_fp = pd.DataFrame(columns=['Driver'] + df_session_cols + ['Total Finish', 'Average Finish'] + df_spread_cols)    
    
    for driver in df['Driver'].values:
        
        # extract each driver and fp session results for the sessions present in the data, add to new_row which will be built out in this for loop
        new_row = df.loc[df.Driver == driver][['Driver'] + sessions].values.tolist()[0]
        
        # add up total win position
        tot = 0
        for session in sessions:
            tot += int(df.loc[df['Driver'] == driver][session].values)
        
        # calculate average win position
        avg = tot/len(sessions)
        
        # calculate session spreads
        fp1_fp2_change = False
        fp2_fp3_change = False
        fp1_fp3_change = False
        
        for session in sessions:
            if 'fp3' in session:
                fp2_fp3_change = int(df.loc[df.Driver == driver][sessions[1]].values) - int(df.loc[df.Driver == driver][sessions[2]].values)
                fp1_fp3_change = int(df.loc[df.Driver == driver][sessions[0]].values) - int(df.loc[df.Driver == driver][sessions[2]].values)
            if 'fp2' in session: 
                fp1_fp2_change = int(df.loc[df.Driver == driver][sessions[0]].values) - int(df.loc[df.Driver == driver][sessions[1]].values)
                
        new_row.extend([tot, avg, fp1_fp2_change, fp2_fp3_change, fp1_fp3_change])

        driver_eval_fp.loc[len(driver_eval_fp)] = new_row
        
    return driver_eval_fp

In [314]:
driver_eval = driver_eval_fp_func(fp_df, 'bahrain')

There are 3 practice sessions in this dataframe for Bahrain in 2024.


In [317]:
driver_eval.sort_values(by=['Total Finish', 'FP3 Results'])

,Driver,FP1 Results,FP2 Results,FP3 Results,Total Finish,Average Finish,FP1-FP2 Change,FP2-FP3 Change,FP1-FP3 Change
8,Alonso,5,3,2,10,3.333333,2,1,3
0,Verstappen,6,6,3,15,5.000000,0,3,3
3,Russel,7,2,6,15,5.000000,5,-4,1
6,Piastri,3,5,7,15,5.000000,-2,-2,-4
5,Sainz,11,4,1,16,5.333333,7,3,10
4,Leclerc,8,9,4,21,7.000000,-1,5,4
2,Hamilton,9,1,12,22,7.333333,8,-11,-3
10,Ricciardo,1,12,11,24,8.000000,-11,1,-10
7,Norris,2,20,5,27,9.000000,-18,15,-3
1,Perez,12,10,8,30,10.000000,2,2,4


In [318]:
def team_eval_fp_func(df, prac):
    '''
    This function evaluates team strength following each free practice session. This is done on several parameters:
    
    Average Finish: average finishing position of the team based on driver positions. A lower value in this column is better.
    Total Finish: sum of the finishing positions of each driver. A lower value in this column is better.
    Finish Spread: difference between finishing positions of the team's drivers. A lower value in this column means the drivers finished closer together on the grid.
    
    Parameters:
        df: the full odds df to be evaluated. *Important* make sure the free practice session to be evaluated has been added to the dataframe prior to running
        prac: the free practice session to be evaluated, named following the format <session>_<race> like 'fp1_bahrain' or 'fp2_spa' etc.
    
    Returns:
        df: this is a dataframe of the results calculated in the function, sorted by Average Finish, then by Total Finish, then by Finish Spread.
    
    '''
    
    team_eval_fp = pd.DataFrame(columns =['Team', 'Average Finish', 'Total Finish', 'Finish Spread'])
    
    for team in df['Team'].drop_duplicates().values:
        team_sum = df.loc[df['Team'] == team][prac].sum()
        team_avg_finish = df.loc[df['Team'] == team][prac].mean()
        team_spread = df.loc[df['Team'] == team][prac].max() - df.loc[df['Team'] == team][prac].min()

        team_eval_fp.loc[len(team_eval_fp.index)] = [team, team_avg_finish, team_sum, team_spread]
        
    return team_eval_fp.sort_values(by=['Average Finish', 'Total Finish', 'Finish Spread']).reset_index(drop=True)

In [319]:
team_eval_fp1 = team_eval_fp_func(fp_df, 'fp1_bahrain')
team_eval_fp1

,Team,Average Finish,Total Finish,Finish Spread
0,McLaren,2.5,5,1
1,RB,2.5,5,3
2,Mercedes,8.0,16,2
3,Red Bull,9.0,18,6
4,Ferrari,9.5,19,3
5,Aston Martin,9.5,19,9
6,Sauber,12.5,25,5
7,Williams,14.5,29,3
8,Alpine,17.5,35,1
9,Haas,19.5,39,1


In [320]:
team_eval_fp2 = team_eval_fp_func(fp_df, 'fp2_bahrain')
team_eval_fp2

,Team,Average Finish,Total Finish,Finish Spread
0,Mercedes,1.5,3,1
1,Aston Martin,5.5,11,5
2,Ferrari,6.5,13,5
3,Red Bull,8.0,16,4
4,Haas,10.5,21,7
5,Williams,12.0,24,2
6,McLaren,12.5,25,15
7,RB,13.5,27,3
8,Alpine,17.0,34,2
9,Sauber,18.0,36,2


In [321]:
team_eval_fp3 = team_eval_fp_func(fp_df, 'fp3_bahrain')
team_eval_fp3

,Team,Average Finish,Total Finish,Finish Spread
0,Ferrari,2.5,5,3
1,Red Bull,5.5,11,5
2,McLaren,6.0,12,2
3,Aston Martin,6.0,12,8
4,Mercedes,9.0,18,6
5,Haas,11.5,23,5
6,RB,12.0,24,2
7,Sauber,16.5,33,1
8,Williams,17.0,34,4
9,Alpine,19.0,38,2
